In [1]:
# As first_reg, but with seeking the difference between the label (i.e. true
# value) and the prediction given by nearest point 'interpolation'.

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = False

In [4]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass
    
    
class NearestInterp(FineGridPredictorMixin, NearestInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass
        
    
# class BilinearDiffNearest(_RegressorBase):
#     def predict_single(self, Xi, y):
#         bilinear = BilinearInterp.predict_single(Xi, y)
#         nearest = NearestInterp.predict_single(Xi, y)
#         return [b - n for b, n in zip(bilinear, nearest)]
            

def gen_one_data_with_offset():
    """Essentially gen_one_data, just gives the offset separately."""
    
    # Random solution to the CH equation
    c = np.random.uniform(0, 10)
    peakon = Peakon(c=c)
    # Random location
    t = np.random.uniform(0, 40)
    x = c * t + np.random.uniform(-3, 3)
    
    # Grids at the location
    cg = coarse_grid((t, x))
    fg = fine_grid((t, x))
    # Features: the solution on the coarse grid
    X = peakon.on_grid(cg)
    # Labels: the difference between the solution on the fine grid and the
    # prediction given by nearest neighbour interpolation
    y = peakon.on_grid(fg)
    # Predict expects to be given a batch of elements, so we have to wrap
    # the input X, y in a list, and then extract the prediction from a list
    # of one element.
    predictor = NearestInterp().predict(input_fn=lambda: ({'X': [X]}, [y]))
    y_prediction = next(predictor)[0]
    
    return X, y, y_prediction
    

def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the feature is the solution on a coarse grid and the label is the 
    difference between solution on a fine grid and the prediction given by
    bilinear interpolation.
    """
    X, y, y_prediction = gen_one_data_with_offset()
    return X, y - y_prediction

In [5]:
### DNN hyperparameters
hidden_units = [1000] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir = './saved_models/sixth_reg/'
model_dir = model_dir_str(model_dir, hidden_units, logits, drop_rate)

dnn = Sequential.create_dnn(hidden_units, logits, drop_rate=drop_rate,
                            model_dir=model_dir, log_steps=log_steps,
                            gradient_clip=gradient_clip)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/sixth_reg/1000x10_121_D04', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7e1cb6e7b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 50
    steps = 10000
    train_input_fn = BatchData(gen_one_data, batch_size)
    dnn.train(input_fn=train_input_fn, max_steps=steps)
    
    batch_size = 100
    steps = 20000
    train_input_fn = BatchData(gen_one_data, batch_size)
    dnn.train(input_fn=train_input_fn, max_steps=steps)
    
    batch_size = 1000
    steps = 25000
    train_input_fn = BatchData(gen_one_data, batch_size)
    dnn.train(input_fn=train_input_fn, max_steps=steps)
    
    batch_size = 5000
    steps = 35000
    train_input_fn = BatchData(gen_one_data, batch_size)
    dnn.train(input_fn=train_input_fn, max_steps=steps)

In [7]:
X, y, y_offset = gen_one_data_with_offset()

In [8]:
result = test_regressors_with_offset([(dnn, True), 
                                      (BilinearInterp(), False), 
                                      (PolyInterp(5), False)], 
                                     lambda: (X, y, y_offset), 
                                    # batch_size=100
                                    )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/sixth_reg/1000x10_121_D04/model.ckpt-32551
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [9]:
result[0].loss, result[1].loss, result[2].loss

(0.3222324901226659, 0.03241407376533531, 0.0020742350714215443)

In [10]:
poly_deg = 5
plot_regressors_with_offset([(Perfect(), False, 'Exact'),
                             (dnn, True, 'DNN'),
                             (BilinearInterp(), False, 'Bilinear'),
                             (PolyInterp(poly_deg), False, 'Poly-{}'.format(poly_deg))],
                            X, y, y_offset)

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/sixth_reg/1000x10_121_D04/model.ckpt-32551
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
